In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
import sys
os.chdir('/gdrive/MyDrive/code/Heart Rate Detection/')
sys.path.insert(1, 'packages')
print(os.listdir())

['HR_learn.ipynb', '__pycache__', 'model_data', 'packages', 'cnn_weights', '37_hr.txt', '1D_cnn.ipynb']


In [3]:
import cv2
import os
import numpy as np
import pandas as pd
import time
from imutils import face_utils
from face_utilities import Face_utilities
from signal_processing import Signal_processing
from evm import *
from lowlight_test import *
import matplotlib.pyplot as plt
from scipy import stats
import tqdm.notebook as tq


## Create Dataframe


In [ ]:
data = [[3, 70],
        [4, 90],
        [6, 85],
        [12, 72],
        [14, 98],
        [15, 83],
        [16, 85],  
        [20, 73],
        [21, 65],
        [22, 80],
        [23, 70],
        [24, 87],
        [25, 80],
        [26, 73],
        [27, 110],
        [28, 90],
        [29, 115],
        [30, 80],
        [31, 80],
        [32, 77],
        [33, 100],
        [34, 91],
        [35, 57],
        [36, 90],
        [37, 90],
        [38, 90],
        [39, 72],
        [40, 85],
        [41, 95],
        [42, 88],
        [43, 65],
        [44, 85],
        [45, 53],
        [46, 78],
        [47, 75],
        [48, 95],
        [49, 90],
        [50, 93],
        [51, 98],
        [52, 100],
        [53, 84],
        [54, 78],
        [55, 117],
        [56, 93],
        [57, 90],
        [58, 93],
        [59, 113],
        [60, 70],
        [61, 82],
        [62, 100]
        ]
df = pd.DataFrame(columns = ['sub', 'HR'], data = data)
print(df.shape)
print(df.head())

(50, 2)
   sub  HR
0    3  70
1    4  90
2    6  85
3   12  72
4   14  98


## HeartRate Class

In [ ]:
class HeartRate():
    def __init__(self, video_path = None, webcam = False, save = False,
                 color_magnify = False, mask_path = None, out_path = None):
        self.video_path = video_path
        #self.video_tensor = video_tensor
        self.webcam = webcam
        self.color_magnify = color_magnify
        self.mask_path = mask_path
        self.out_path = out_path
        self.save = save
        self.out = None
    
    def run(self):
        video = False
        if self.video_path is not None and self.color_magnify is False:
            #video = True
            fr = 0
            cap = cv2.VideoCapture(self.video_path)
        elif self.webcam:
            cap = cv2.VideoCapture(0)                        
        else:
            pass
            #cap = cv2.VideoCapture(0)
            #fr = 0
            
        fu = Face_utilities()
        sp = Signal_processing()
        evm = EVM()
        
        i=0
        last_rects = None
        last_shape = None
        last_age = None
        last_gender = None
        
        face_detect_on = False
        age_gender_on = False
    
        t = time.time()
        
        #for signal_processing
        BUFFER_SIZE = 100
        BPM_BUFFER_SIZE = 100
        

        times = []
        data_buffer = []
        bpm_buffer = []
        
        # data for plotting
        filtered_data = []
        
        fft_of_interest = []
        freqs_of_interest = []

        norm_data = []
        fft_data = []
        freq_data = []
        
        bpm = 0
        
        if self.color_magnify:
            print('Magnifying color using EVM')
            sub = self.video_path.split('/')[-1].split('.')[-2]
            cm_out_path = os.path.join('/gdrive/MyDrive/stable videos/ISI/color_magnify/', sub + '_cm.avi')
            print(cm_out_path)
            video_tensor = evm.magnify_color(video_name = self.video_path, mask = self.mask_path, out_name = cm_out_path,
                     low = 1, high = 1.7, save = True, skin_detect = False, inpaint_bg = False, amplification= 15)

            cap = cv2.VideoCapture(cm_out_path)


        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("Total frames: ", total_frames)    
        frames_processed = 0

        
        for i in tq.tqdm(range(total_frames)):
            # grab a frame -> face detection -> crop the face -> 68 facial landmarks -> get mask from those landmarks
    
            # calculate time for each loop
            t0 = time.time()
            
            if(i%1==0):
                face_detect_on = True
                if(i%10==0):
                    age_gender_on = True
                else:
                    age_gender_on = False
            else: 
                face_detect_on = False
            
            if self.video_path:
                ret, frame = cap.read()
                #print("frame shape: ", frame.shape)
            else:
                frame = video_tensor[fr]
                fr += 1

            
            
            ret_process = fu.no_age_gender_face_process(frame, "68")
            frames_processed += 1 
            #print('Frames Processed: ', frames_processed)

            if ret_process is None:
                continue
            
            rects, face, shape, aligned_face, aligned_shape = ret_process
            
#            (x, y, w, h) = face_utils.rect_to_bb(rects[0])
            (x, y, w, h) = face_utils.rect_to_bb(rects)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

            
            if(len(aligned_shape)==68):
                cv2.rectangle(aligned_face,(aligned_shape[54][0], aligned_shape[29][1]), #draw rectangle on right and left cheeks
                        (aligned_shape[12][0],aligned_shape[33][1]), (0,255,0), 0)
                cv2.rectangle(aligned_face, (aligned_shape[4][0], aligned_shape[29][1]), 
                        (aligned_shape[48][0],aligned_shape[33][1]), (0,255,0), 0)
            else:
                cv2.rectangle(aligned_face, (aligned_shape[0][0],int((aligned_shape[4][1] + aligned_shape[2][1])/2)),
                            (aligned_shape[1][0],aligned_shape[4][1]), (0,255,0), 0)
                
                cv2.rectangle(aligned_face, (aligned_shape[2][0],int((aligned_shape[4][1] + aligned_shape[2][1])/2)),
                            (aligned_shape[3][0],aligned_shape[4][1]), (0,255,0), 0)
            
            for (x, y) in aligned_shape: 
                cv2.circle(aligned_face, (x, y), 1, (0, 0, 255), -1)
                
                
            #for signal_processing
            ROIs = fu.ROI_extraction(aligned_face, aligned_shape)
            green_val = sp.extract_color(ROIs)
            #print(green_val)
            
            data_buffer.append(green_val)
            
            if(video==False):
                times.append(time.time() - t)
            else:
                times.append((1.0/298.)*i)
                pass
            
            L = len(data_buffer)
            c = 0
            
            if L > BUFFER_SIZE:
                data_buffer = data_buffer[-BUFFER_SIZE:]
                #print("buffer length: ", len(data_buffer))
                # print(data_buffer)

                times = times[-BUFFER_SIZE:]
                #bpms = bpms[-BUFFER_SIZE//2:]
                L = BUFFER_SIZE
            #print(times)
            if L==100:
                c += 1 
                fps = float(L) / (times[-1] - times[0])
                cv2.putText(frame, "fps: {0:.2f}".format(fps), (30,int(frame.shape[0]*0.05)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
                #print("buffer length2: ", len(data_buffer))
                #whole_data.append(data_buffer)
                detrended_data = sp.signal_detrending(data_buffer)
                #print("Detrended Data:", detrended_data)
                #print("Len: ", len(detrended_data))
                #print(len(times))
                
                try:
                    interpolated_data = sp.interpolation(detrended_data, times)
                except:
                    interpolated_data = detrended_data
                    
                
                smoothed_data = sp.median_filter(interpolated_data, 15)
                rm = sp.running_mean(interpolated_data, 3)
                
                
                normalized_data = sp.normalization(interpolated_data)
                normalized_data_med = sp.normalization(smoothed_data)
                #normalized_data_rm = sp.normalization(rm)

                norm_data.append(normalized_data_med)
                
                fft_of_interest, freqs_of_interest = sp.fft(normalized_data, fps)
                fft_of_interest_med, freqs_of_interest_med = sp.fft(normalized_data_med, fps)
                #fft_of_interest, freqs_of_interest = sp.fft(normalized_data, fps)

                fft_data.append(fft_of_interest_med)
                freq_data.append(freqs_of_interest_med)
                
                
                """ For Hueristic approach of HR detection"""
                # max_arg = np.argmax(fft_of_interest)
                # max_arg_med = np.argmax(fft_of_interest_med)
                
                # bpm = freqs_of_interest[max_arg]
                # bpm_med = freqs_of_interest_med[max_arg_med]
                
                # bpm_buffer.append(bpm_med)
                # if len(bpm_buffer) > BPM_BUFFER_SIZE:
                #     bpm_med = stats.mode(bpm_buffer[-BPM_BUFFER_SIZE:])[0][0]
                    

            if self.save:
                if self.out is None:
                    self.out = cv2.VideoWriter(self.out_path, -1, 20.0, (frame.shape[1],frame.shape[0]))
            if self.save and frame is not None:
                self.out.write(frame)
            
            #cv2.imshow("mask",mask)
            i = i+1

        if self.save:
            self.out.release()

        return norm_data, fft_data, freq_data




## Main

In [ ]:
def main():
  vid_dir = '/gdrive/MyDrive/stable videos/ISI/normal_light/'
  for subject in df['sub'].tolist():
    print('Processing: ' + 'sub_' + str(subject) + '.mp4')

    video_path = os.path.join(vid_dir, 'sub_' + str(subject) + '.mp4')
    out_path = os.path.join('/gdrive/MyDrive/stable videos/ISI/df/', 'sub_' + str(subject) + '.pkl')

    hr_gt = df[df['sub'] == subject]['HR'].values[0]

    color_magnify = True

    hr = HeartRate(video_path = video_path, save = False, color_magnify = color_magnify,
                           mask_path = None, out_path = None)

    norm_data, fft_data, freq_data = hr.run()

    hr_gt = pd.DataFrame(columns = ['HR'], data = [hr_gt for _ in range(len(norm_data))])

    norm_data = pd.DataFrame(data = norm_data)
    fft_data = pd.DataFrame(data = fft_data)
    freq_data = pd.DataFrame(data = freq_data)

    norm_data = pd.concat([norm_data, hr_gt], axis = 1)
    fft_data = pd.concat([fft_data, hr_gt], axis = 1)
    freq_data = pd.concat([freq_data, hr_gt], axis = 1)


    if color_magnify:
      norm_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/with_color_mag/norm_data/', 'sub_' + str(subject) + '_norm_data.pkl'))
      fft_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/with_color_mag/fft_data/', 'sub_' + str(subject) + '_fft_data.pkl'))
      freq_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/with_color_mag/freq_data/', 'sub_' + str(subject) + '_freq_data.pkl'))
    
    else:
      norm_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/without_color_mag/norm_data/', 'sub_' + str(subject) + '_norm_data.pkl'))
      fft_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/without_color_mag/fft_data/', 'sub_' + str(subject) + '_fft_data.pkl'))
      freq_data.to_pickle(os.path.join('/gdrive/MyDrive/stable videos/ISI/df/normal_light/without_color_mag/freq_data/', 'sub_' + str(subject) + '_freq_data.pkl'))




        
        

if __name__ == "__main__":
  main()


Processing: sub_3.mp4
Magnifying color using EVM
/gdrive/MyDrive/stable videos/ISI/color_magnify/sub_3_cm.avi
Magnifying Color..
origin video shape:  (926, 280, 400, 3)
final video shape:  (926, 35, 50, 3)
saving magnified video
Total frames:  926


[INFO] load 68 facial landmarks model ...
[INFO] Load model - DONE!


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Processing: sub_4.mp4
Magnifying color using EVM
/gdrive/MyDrive/stable videos/ISI/color_magnify/sub_4_cm.avi
Magnifying Color..
origin video shape:  (885, 280, 400, 3)
final video shape:  (885, 35, 50, 3)
saving magnified video
Total frames:  885


[INFO] load 68 facial landmarks model ...
[INFO] Load model - DONE!


...

[INFO] load 68 facial landmarks model ...
[INFO] Load model - DONE!


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Processing: sub_40.mp4
Magnifying color using EVM
/gdrive/MyDrive/stable videos/ISI/color_magnify/sub_40_cm.avi
Magnifying Color..
origin video shape:  (1085, 280, 400, 3)
final video shape:  (1085, 35, 50, 3)
saving magnified video
Total frames:  1085


[INFO] load 68 facial landmarks model ...
[INFO] Load model - DONE!


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
len(freq_data[12])


22

In [ ]:
pd.DataFrame(data = fft_data).tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
683,24.119149,6.981058,10.480645,24.436200,16.087430,3.310144,2.591593,4.566252,1.562177,5.514574,9.289498,2.916682,3.265028,0.644042,1.762834,2.763905,0.574655,2.174849,1.164263,1.257117,1.612093,2.413946,NaN
684,24.130515,4.455731,6.939109,18.380800,17.251608,4.042673,1.021598,5.054288,1.331686,2.134850,9.886628,4.608594,1.079791,0.295837,1.005744,2.411664,0.066668,0.804410,2.685707,1.123409,1.048936,4.455245,NaN
685,24.898095,3.769771,5.268166,12.272665,15.198010,7.685559,0.495551,4.216467,2.750319,1.048091,6.464330,5.333414,0.923849,0.194799,1.690343,0.632774,0.010479,0.075716,1.651528,2.887257,0.485893,3.041600,NaN
686,17.222119,3.048308,7.831702,6.567249,10.530406,13.172604,1.035793,2.197349,4.992479,1.995782,2.624743,2.874746,1.089763,0.879079,1.097671,0.116453,0.575774,0.917781,0.722983,2.446763,1.470221,1.679414,NaN
687,17.180031,2.271660,12.558099,4.373052,5.702755,15.337779,1.431971,0.605618,3.985323,3.279270,2.087652,1.653339,2.331762,0.319847,0.182519,0.622311,0.931927,1.224461,1.432019,0.643835,1.091546,3.135423,NaN


In [ ]:
spd.DataFrame(data = fft_data).drop(100,1).tail()